### Env 실험 및 영상 저장을 위한 노트북

(gym_env_test_plot 노트북 참조)

In [ ]:
import horcrux_terrain_v2
import gymnasium as gym
import numpy as np
import pandas as pd

import os
import pathlib
import time
import matlab.engine
import mediapy as media

from scipy.ndimage import uniform_filter1d
from scipy.spatial.transform import Rotation

import matplotlib.pyplot as plt

from gymnasium.utils.save_video import save_video

from IPython.display import Video

### 필요 함수 정의

In [ ]:
from re import I


def get_unique_filename(base_path, ext=".mp4"):
    """중복된 파일명이 존재하면 숫자를 증가하여 새로운 경로를 반환"""
    if not base_path.endswith(ext):
        base_path += ext  # 확장자 자동 추가

    file_name, file_ext = os.path.splitext(base_path)  # 파일명과 확장자 분리
    count = 0
    new_path = f"{file_name}-episode-0"+file_ext

    while os.path.exists(new_path):  # 파일 존재 여부 확인
        new_path = f"{file_name}{count}-episode-0{file_ext}"
        count += 1


    return f"rl-video{count-1}", new_path

def default_plot(x, y, f_name='default_plot', legends=['acc_x', 'acc_y', 'acc_z'], title=''):
    colors = plt.get_cmap("tab10").colors
    fig, ax = plt.subplots(figsize=(15/2.54, 10/2.54))
    ax.set_facecolor((0.95, 0.95, 0.95)) 

    n_column = len(np.shape(y))
    if n_column>2:
        print("The dimmension of data must be less than 3. (1D or 2D)")
        return -1
    
    n_data = np.shape(y)[1]

    for i in range(n_data):
        # **Plot**
        ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[i], label=legends[i])
        # ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[1], label=legends[1])
        # ax.plot(x, y[:,i], linewidth=1.5, linestyle="-", color=colors[2], label=legends[2])

    # **Grid 설정**
    ax.grid(True, linestyle="--", linewidth=1, color="#202020", alpha=0.7)  # 주요 그리드
    ax.minorticks_on()
    ax.grid(True, which="minor", linestyle=":", linewidth=0.5, color="#404040", alpha=0.5)  # 보조 그리드

    # **Axis 스타일 설정**
    ax.spines["top"].set_linewidth(1.0)
    ax.spines["right"].set_linewidth(1.0)
    ax.spines["left"].set_linewidth(1.0)
    ax.spines["bottom"].set_linewidth(1.0)

    ax.tick_params(axis="both", labelsize=11, width=1.0)  # 폰트 크기 및 라인 두께
    ax.xaxis.label.set_size(12)
    ax.yaxis.label.set_size(12)

    # **폰트 및 제목 설정**
    plt.rcParams["font.family"] = "Arial"
    ax.set_xlabel("X-Axis", fontsize=12, fontweight="bold")
    ax.set_ylabel("Y-Axis", fontsize=12, fontweight="bold")
    ax.set_title(title, fontsize=14, fontweight="bold")

    # **Legend (MATLAB 스타일 적용)**
    ax.legend(loc="upper right", ncol=3, fontsize=10, frameon=True)

    # **비율 설정 (MATLAB의 `pbaspect([2.1 1 1])`과 비슷한 효과)**
    fig.set_size_inches(2.1 * 5, 5)  # 비율 2.1:1 (기본 높이 5inch 기준)

    # **Save Figure (MATLAB saveas와 유사)**
    plt.savefig(f"./figs/{f_name}.png", dpi=600, bbox_inches="tight")

    plt.show()

def moving_average(data, window_size):
    kernel = np.ones(window_size) / window_size
    return np.convolve(data, kernel, mode='same')  # 'valid'는 경계 제외

ENV 생성

In [ ]:
env_config = {
    "forward_reward_weight": 6.5,
    "side_cost_weight": 2.0,
    "unhealthy_max_steps": 100,
    "healthy_reward": 0.5,
    "healthy_roll_range": (-35,35),
    "terminating_roll_range": (-85,85),
    "rotation_norm_cost_weight": 0.01,
    "rotation_orientation_cost_weight": 1.2,
    "termination_reward": 0,
    "gait_params": (30, 30, 60, 60, 0),
    "use_friction_chg": False,
    "joy_input_random": True,
    "render_mode": "rgb_array",
    "render_camera_name": 'ceiling',
    "use_imu_window": False,
}

env = gym.make("horcrux_terrain_v2/plane-v2", **env_config)

동작 시작 및 영상 저장

In [ ]:
env.reset()

env_done = False

_video_base_name = 'rl-video'

frames = []
info = []

for i in range(1000):
    _, _, _, env_done, env_info = env.step([0.75]*14)
    pixels = env.render()
    frames.append(pixels)
    info.append(env_info)

_f_name, _full_path = get_unique_filename(f"./video/{_video_base_name}")
save_video(frames, "./video/", name_prefix=_f_name, fps=env.metadata['render_fps'])

Video(_full_path, embed=True, width=700)

관련 INFO 데이터 추출

In [ ]:
_len_info = len(info)

"""
"head_quat": observation[-27:-23].copy(),
"head_ang_vel": observation[-23:-20].copy(),
"head_lin_acc": observation[-20:-17].copy(),
"motion_vector": observation[-17:-3].copy(),
"""

_head_quat_array = np.array([_info['head_quat'] for _info in info])
_head_ang_vel_array = np.array([_info['head_ang_vel'] for _info in info])
_head_lin_acc_array = np.array([_info['head_lin_acc'] for _info in info])
_motion_vector_array = np.array([_info['motion_vector'] for _info in info])

_com_xy_vel = np.array([[_info['x_velocity'], _info['y_velocity']] for _info in info])
_com_yaw_vel = np.array([_info['yaw_velocity'] for _info in info])
_com_vels = np.hstack((_com_xy_vel, _com_yaw_vel.reshape(-1,1)))

_ftd_com_vels = uniform_filter1d(_com_vels, axis=0, size=5)

_head_euler_array_in = Rotation.from_quat(_head_quat_array, scalar_first=True).as_euler('ZYX', degrees=True)
_ftd_head_euler_array_in = uniform_filter1d(_head_euler_array_in, axis=0, size=10)
ftd_lin_acc_array = uniform_filter1d(_head_lin_acc_array, size=10, axis=0)
_head_lin_acc_array[0:10,0]

In [ ]:
x = np.linspace(0, _len_info/10, _len_info)

default_plot(x, _head_quat_array,f_name='head_quat_movfiltered_w200', legends=['w','x', 'y', 'z'])
default_plot(x, _head_euler_array_in, f_name='head_euler_movfiltered_w200', title='Intrinsic Rotation',legends=['yaw', 'pitch', 'roll'])
# default_plot(x, _ftd_head_euler_array_in, f_name='head_euler_int_movmean2', title='Intrinsic Rotation (Filtered)',legends=['yaw', 'pitch', 'roll'])
# default_plot(x, _ftd_com_vels, f_name='ftd_com_xy_vels', title='CoM XY Velocity',legends=['x', 'y', 'yaw'])
# default_plot(x, _com_vels, f_name='com_xy_vels', title='CoM XY Velocity',legends=['x', 'y', 'yaw'])

In [ ]:
# from scipy.io import savemat
# savemat('./data/rl_data.mat', {'raw_data': _head_lin_acc_array, 'ftd_data': ftd_lin_acc_array})